Yessir 💥  
This is where we level up from toy models to **transfer learning** — fine-tuning pretrained models on **your own data**, even with **limited compute**.

---

# 🧪 `09_lab_finetune_resnet_on_custom_data.ipynb`  
### 📁 `02_computer_vision`  
> Take a pretrained **ResNet**, swap the head, and fine-tune it on a small dataset like **Flowers**, **Food101**, or your own folder of labeled images.

---

## 🎯 Learning Objectives

- Understand **feature extraction vs full fine-tuning**  
- Use **Torchvision’s ResNet18** pretrained on ImageNet  
- Apply on a **small dataset subset** (Colab/laptop safe)  
- Visualize **accuracy curves & confusion matrix**

---

## 💻 Runtime Design

| Spec                | Setting            |
|---------------------|--------------------|
| Dataset             | Flowers/CIFAR or local custom  
| Hardware            | CPU / T4 GPU (Colab)  
| RAM                 | < 3GB  
| Epochs              | 5–10 for demo  
| Backend             | PyTorch + TorchVision  

---

## 🧱 Section 1: Imports

```python
import torch
import torchvision
import torchvision.transforms as transforms
import torchvision.models as models
import torch.nn as nn
import torch.optim as optim
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import os
```

---

## 🖼️ Section 2: Dataset Prep (Flowers or Custom Folder)

### Option A: Flowers (Auto-downloadable)

```python
# Flowers subset (~600 images)
!wget https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz
!tar -xf flower_photos.tgz

data_dir = 'flower_photos'
```

### Option B: Your own data (Structure it like)

```
my_dataset/
├── class1/
│   ├── img1.jpg
│   └── ...
├── class2/
│   └── ...
```

---

## 🧪 Section 3: Transforms + Loaders

```python
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

dataset = ImageFolder(data_dir, transform=transform)
train_size = int(0.8 * len(dataset))
train_set, val_set = torch.utils.data.random_split(dataset, [train_size, len(dataset) - train_size])

train_loader = DataLoader(train_set, batch_size=16, shuffle=True)
val_loader = DataLoader(val_set, batch_size=16, shuffle=False)
```

---

## 🤖 Section 4: Load & Modify ResNet18

```python
model = models.resnet18(pretrained=True)

# Freeze early layers
for param in model.parameters():
    param.requires_grad = False

# Replace final layer
num_classes = len(dataset.classes)
model.fc = nn.Linear(model.fc.in_features, num_classes)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
```

---

## ⚙️ Section 5: Train

```python
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.fc.parameters(), lr=0.001)

train_loss, val_loss = [], []

for epoch in range(5):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    train_loss.append(running_loss / len(train_loader))

    # Validation
    model.eval()
    running_val_loss = 0.0
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            running_val_loss += loss.item()
    val_loss.append(running_val_loss / len(val_loader))

    print(f"Epoch {epoch+1}: Train Loss {train_loss[-1]:.4f}, Val Loss {val_loss[-1]:.4f}")
```

---

## 📊 Section 6: Plot Loss Curves

```python
plt.plot(train_loss, label='Train')
plt.plot(val_loss, label='Validation')
plt.legend()
plt.title("Training vs Validation Loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.grid(True)
plt.show()
```

---

## 📉 Section 7: Evaluate (Optional Confusion Matrix)

```python
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import numpy as np

model.eval()
all_preds, all_labels = [], []

with torch.no_grad():
    for inputs, labels in val_loader:
        inputs = inputs.to(device)
        outputs = model(inputs)
        preds = outputs.argmax(dim=1).cpu().numpy()
        all_preds.extend(preds)
        all_labels.extend(labels.numpy())

cm = confusion_matrix(all_labels, all_preds)
disp = ConfusionMatrixDisplay(cm, display_labels=dataset.classes)
disp.plot(cmap='viridis', xticks_rotation=45)
plt.title("Validation Confusion Matrix")
plt.grid(False)
plt.show()
```

---

## ✅ Lab Recap

| Concept                         | Covered |
|---------------------------------|---------|
| Transfer learning (ResNet18)    | ✅       |
| Feature extraction vs fine-tune | ✅       |
| Small dataset training          | ✅       |
| Visual evaluation               | ✅       |
| Colab- and laptop-ready         | ✅       |

---

## 🧠 What You Learned

- How to **repurpose pretrained models** for your own tasks  
- The role of **freezing layers** vs retraining  
- How to structure and run **vision fine-tuning pipelines**

---

You want to move to NLP labs next (`03_natural_language_processing`) or finish up more vision-style labs before that?